# - 본격적 60개 컬럼 스케일링 (Pipeline | ColumnTransformer)
# - 다중 공선성 VIF
# - fit / predict (기존 모델 + XGBoost | LightGBM)
# - 교차검증
# - 결과확인

In [1]:
pip install xgboost lightgbm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 파일 불러오기

In [2]:
import pandas as pd

In [3]:
filepath = r"/Users/seoyeongbubu/jump-to-github/jump-to-github/data/merged_sum.csv"

In [4]:
df = pd.read_csv(filepath)

In [5]:
df

,구,연도,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,...,증감률,1인가구_비율,20세 미만,65세 이상,0~14세,소비자물가,저소득노인_65~79비율,저소득노인_80이상비율,기초생활수급자총인원,기초생활수급자비율
0,강남구,2017,1,0,0,0,0,0,0,0,...,1.3,12.36,539,7607,68591,97.645,4.64,6.81,1706,22.43
1,강남구,2018,1,0,0,0,0,0,0,0,...,-0.1,12.00,421,7834,66657,99.086,45.66,36.18,1840,23.49
2,강남구,2019,1,0,0,0,0,0,0,0,...,5.3,12.98,607,8393,66767,99.466,1.68,4.44,2959,35.26
3,강남구,2020,1,0,0,0,0,0,0,0,...,4.4,13.58,498,9162,65904,100.000,8.87,7.07,3994,43.59
4,강남구,2021,1,0,0,0,0,0,0,0,...,4.3,14.31,397,10292,64520,102.500,2.19,5.48,3970,38.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,중랑구,2019,0,0,0,0,0,0,0,0,...,6.1,13.51,201,10920,37243,99.466,5.54,20.87,4826,44.19
171,중랑구,2020,0,0,0,0,0,0,0,0,...,8.4,14.68,206,12190,35444,100.000,4.07,16.39,6275,51.48
172,중랑구,2021,0,0,0,0,0,0,0,0,...,6.9,15.90,169,13358,33325,102.500,9.10,31.97,7074,52.96
173,중랑구,2022,0,0,0,0,0,0,0,0,...,6.4,16.97,213,14569,32188,107.720,2.93,7.34,7778,53.39


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 48 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   구              175 non-null    object 
 1   연도             175 non-null    int64  
 2   강남구            175 non-null    int64  
 3   강동구            175 non-null    int64  
 4   강북구            175 non-null    int64  
 5   강서구            175 non-null    int64  
 6   관악구            175 non-null    int64  
 7   광진구            175 non-null    int64  
 8   구로구            175 non-null    int64  
 9   금천구            175 non-null    int64  
 10  노원구            175 non-null    int64  
 11  도봉구            175 non-null    int64  
 12  동대문구           175 non-null    int64  
 13  동작구            175 non-null    int64  
 14  마포구            175 non-null    int64  
 15  서대문구           175 non-null    int64  
 16  서초구            175 non-null    int64  
 17  성동구            175 non-null    int64  
 18  성북구       

In [7]:
df.shape

(175, 48)

In [8]:
# '구' 원핫 인코딩 한 컬럼이 존재하므로 문자열인 기존 '구'컬럼은 삭제한다.
df = df.drop(columns=["구"])

In [9]:
# DataFrame 확인하기
df.shape

(175, 47)

### Target(y), feature(X) 

In [10]:
target_col = "값"
y = df[target_col]

In [11]:
# feature는 target으로 잡은 컬럼을 제외한 나머지
X = df.drop(columns=[target_col]) 

In [12]:
# 수치형 컬럼과 범주형 컬럼을 나누기위해 전체 컬럼 출력 후 확인하기
df.columns

Index(['연도', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구',
       '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구', '단독주택_계', '아파트_계', '연립주택_계',
       '다세대주택_계', '비주거용 건물내 주택_계', '값', '노령화지수', '총인구', '총인구_남자', '총인구_여자',
       '전년 대비 증감', '증감률', '1인가구_비율', '20세 미만', '65세 이상', '0~14세', '소비자물가',
       '저소득노인_65~79비율', '저소득노인_80이상비율', '기초생활수급자총인원', '기초생활수급자비율'],
      dtype='object')

In [13]:
# 원핫 인코딩 컬럼 가져오기
region_cols = [
    '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
    '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구',
    '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'
]

In [14]:
# 수치형 컬럼 = 전체 컬럼 - 원핫 인코딩 컬럼
numeric_cols = [c for c in X.columns if c not in region_cols]

### 학습/테스트 분할

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
# 컬럼이 많으므로 학습/테스트 데이터 크기 확인해보기
print("X_train :", X_train.shape)
print("X_test :", X_test.shape)

X_train : (140, 46)
X_test : (35, 46)


### Pipeline과 ColumnTransformer로 전처리 및 스케일링

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [19]:
col_trans = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols), # 수치형(numeric)만 스케일링
        ("cat", "passthrough", region_cols) # 범주형(ohe)은 통과
    ]
)

In [20]:
# 전처리 할 파이프라인 생성
preprocess_pipeline = Pipeline([("preprocess", col_trans)])

In [21]:
# Pipeline으로 스케일링 한 데이터를 학습시키기
X_train_scaled = preprocess_pipeline.fit_transform(X_train)

X_test_scaled = preprocess_pipeline.transform(X_test) # test는 transform만

In [22]:
# 전처리 완료 된 수치형 컬럼과 원핫 인코딩 컬럼을 합쳐서 새로운 컬럼명으로 묶기
scaled_feature_names = ([f"scaled_{c}" for c in numeric_cols] + region_cols)

print("[전처리 후 X_train_scaled]", X_train_scaled.shape)

[전처리 후 X_train_scaled] (140, 46)


## 다중 공선성 VIF

### 다중 공선성(VIF) : 서로 비슷한 의미를 가진 컬럼이 많으면 모델이 학습할 때 혼란스러워지는 현상을 잡아내는 도구

In [23]:
pip install statsmodels


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [25]:
# 컬럼은 전처리 한 scaled_feature_names를 사용한다
df_vif = pd.DataFrame(X_train_scaled, columns=scaled_feature_names)
df_vif.shape

(140, 46)

In [26]:
vif = pd.DataFrame()
vif["feature"] = scaled_feature_names
vif["VIF"] = [
    variance_inflation_factor(df_vif.values, i)
    for i in range(df_vif.shape[1])
]

print("\n===== [VIF 결과] =====")
print(vif.sort_values("VIF", ascending=False))


===== [VIF 결과] =====
                 feature          VIF
8          scaled_총인구_남자          inf
9          scaled_총인구_여자          inf
7             scaled_총인구          inf
15          scaled_0~14세  1049.269873
1          scaled_단독주택_계   685.890892
5   scaled_비주거용 건물내 주택_계   464.925624
14         scaled_65세 이상   288.412738
2           scaled_아파트_계   223.976301
38                   송파구   222.576308
44                    중구   217.047789
19     scaled_기초생활수급자총인원   192.376088
4         scaled_다세대주택_계   171.455632
12        scaled_1인가구_비율   153.255904
43                   종로구   141.996745
25                   관악구   114.579242
24                   강서구   101.064170
29                   노원구    94.565980
41                   용산구    68.790340
0              scaled_연도    65.756966
6           scaled_노령화지수    64.442799
21                   강남구    59.425314
27                   구로구    47.420332
28                   금천구    43.770047
20      scaled_기초생활수급자비율    42.488449
42                   은평구    

/Users/seoyeongbubu/Desktop/Kg/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


### VIF 값을 보면 공선성을 띄는 컬럼들이 많은것을 확인 할 수 있다.


### 컬럼의 유형별로 1 ~ 2개만 남기고 중복되는 의미의 컬럼은 삭제해서 다시 feature정의한다.

In [27]:
numeric_keep = [
    "연도",
    "총인구",
    "20세 미만",
    "65세 이상",
    "단독주택_계",
    "아파트_계",
    "기초생활수급자비율",
    "저소득노인_65~79비율",
    "저소득노인_80이상비율",
    "1인가구_비율",
    "소비자물가",
    "증감률"
]

In [28]:
# '중구'가 빠진 이유는 원핫 인코딩 특성상 다 더하면 1이므로 완전하게 선형을 이루므로 1개를 제거한다.
region_keep = [
    '강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구',
    '노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구',
    '성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중랑구'
]

In [29]:
# 타깃과 피처 재 정의하기
final_features = numeric_keep + region_keep

target_col = "값"
y = df[target_col]
X = df[final_features]

print("[최종 선택된 피처 개수]", len(final_features))

[최종 선택된 피처 개수] 36


### 학습/테스트 분할하기

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [32]:
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

X_train: (140, 36)
X_test : (35, 36)


### 최종적인 컬럼을 다시 전처리하기

### 전처리를 왜 다시할까?

### 모델이 vif로 걸러낸 정확한 컬럼의 값으로 예측해야하기 때문이다.

In [33]:
col_trans_final = ColumnTransformer([
    ("num", StandardScaler(), numeric_keep),
    ("cat", "passthrough", region_keep)
])

In [34]:
# 다시 Pipeline으로 전처리하기
pre_final = Pipeline([("pre", col_trans_final)])

In [35]:
X_train_scaled = pre_final.fit_transform(X_train)
X_test_scaled  = pre_final.transform(X_test)

In [36]:
print(X_train_scaled.shape)

(140, 36)


## 모델 학습 적용

In [37]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [38]:
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=0.01),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=42),
    "XGBoost": XGBRegressor(
        n_estimators=300, learning_rate=0.05,
        max_depth=4, subsample=0.9,
        colsample_bytree=0.9,
        random_state=42, objective="reg:squarederror",
        tree_method="hist"
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=300, learning_rate=0.05,
        subsample=0.9, colsample_bytree=0.9,
        random_state=42
    )
}

In [39]:
results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)

    pred = model.predict(X_test_scaled)

    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)

    results[name] = {"mae": mae, "r2": r2}

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000192 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 486
[LightGBM] [Info] Number of data points in the train set: 140, number of used features: 12
[LightGBM] [Info] Start training from score 22.035714
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/Users/seoyeongbubu/Desktop/Kg/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


## gain: -inf 오류발생 >>>> 문제점 발생

### LightGBM 모델이 나눌(split) 만한 정보가 없기 때문에 위처럼 오류메세지가 연속적으로 나오는것.

### 데이터가 너무 적거나, 피처가 너무 단순하거나, 선형적 구조가 강할 때 오류가 발생한다.

### 그렇기 때문에 트리구조에서 depth = 1 ~ 2 에서 멈춘다.

In [40]:
print(f"MAE : {mae:.3f}")
print(f"R²  : {r2:.3f}")

results[name] = {"mae": mae, "r2": r2}

MAE : 3.839
R²  : 0.725


In [41]:
for name, m in results.items():
    print(f"{name:10s} | MAE: {m['mae']:.3f} | R2: {m['r2']:.3f}")

Linear     | MAE: 2.724 | R2: 0.870
Ridge      | MAE: 2.692 | R2: 0.869
RandomForest | MAE: 3.794 | R2: 0.627
XGBoost    | MAE: 3.086 | R2: 0.764
LightGBM   | MAE: 3.839 | R2: 0.725


### 최종 성능 값을 보면 기본 선형회귀 모델보다 트리계열 모델의 값이 낮은것을 볼 수 있다.

## 데이터의 전체적인 안정성을 확인 했으므로 모델을 선정하여
## v.0.4 에서는 아래와 같은 기능을 추가 할 예정이다.

### 1. 연도 기반 Trend Feature 추가

### 2. LagFeature (전년도 값) 생성 > 고독사의 가장 강력한 예측 변수는 전연도 고독사 값이다

### 3. Rolling Mean (3년 평균)

### 4. 비율 피처 * 인구 조합

### 5. L1 규제 (Lasso) > 공선성 완화

### 6. 선형 모델 + XGBoost 모델 최적화 > HyperParam 튜닝